In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [2]:
base_path = r"C:\Users\HP\Downloads\Fitness_Classes_Data\Fitness Classes Data"

df_apr_may = pd.read_csv(base_path + r"\Classes April-May 2018.csv")
df_june = pd.read_csv(base_path + r"\Classes June 2018.csv")


In [3]:
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

df_apr_may = clean_columns(df_apr_may)
df_june = clean_columns(df_june)


In [4]:
critical_cols = ["maxbookees", "number_booked", "price_(inr)"]

df_apr_may = df_apr_may.dropna(subset=critical_cols)
df_june = df_june.dropna(subset=critical_cols)

df_apr_may["activitydescription"] = df_apr_may["activitydescription"].fillna("Unknown")
df_june["activitydescription"] = df_june["activitydescription"].fillna("Unknown")


In [5]:
df_apr_may = df_apr_may.drop_duplicates()
df_june = df_june.drop_duplicates()


In [6]:
num_cols = ["maxbookees", "number_booked", "price_(inr)"]

df_apr_may[num_cols] = df_apr_may[num_cols].apply(pd.to_numeric, errors="coerce")
df_june[num_cols] = df_june[num_cols].apply(pd.to_numeric, errors="coerce")

df_apr_may = df_apr_may.dropna(subset=num_cols)
df_june = df_june.dropna(subset=num_cols)


In [16]:
df_apr_may["bookingenddatetime_(month_/_day_/_year)"] = pd.to_datetime(
    df_apr_may["bookingenddatetime_(month_/_day_/_year)"], errors="coerce"
)

df_june["bookingenddatetime_(month_/_day_/_year)"] = pd.to_datetime(
    df_june["bookingenddatetime_(month_/_day_/_year)"], errors="coerce"
)

df_apr_may["bookingstarttime"] = pd.to_datetime(
    df_apr_may["bookingstarttime"], errors="coerce"
).dt.time

df_june["bookingstarttime"] = pd.to_datetime(
    df_june["bookingstarttime"], errors="coerce"
).dt.time


In [8]:
merge_keys = ["activitysiteid", "activitydescription"]

df_merged = pd.merge(
    df_apr_may,
    df_june,
    on=merge_keys,
    how="outer",
    suffixes=("_apr_may", "_june")
)


In [9]:
date_cols = [c for c in df_merged.columns if "bookingenddatetime" in c]
for col in date_cols:
    df_merged[col] = pd.to_datetime(df_merged[col], errors="coerce")

num_cols_merged = df_merged.select_dtypes(include="number").columns
df_merged[num_cols_merged] = df_merged[num_cols_merged].apply(pd.to_numeric, errors="coerce")


In [10]:
quality_report = pd.DataFrame({
    "check": [
        "total_records",
        "total_missing_values",
        "duplicate_records",
        "overbooked_apr_may",
        "overbooked_june"
    ],
    "value": [
        len(df_merged),
        df_merged.isnull().sum().sum(),
        df_merged.duplicated().sum(),
        (df_merged["number_booked_apr_may"] > df_merged["maxbookees_apr_may"]).sum()
        if {"number_booked_apr_may","maxbookees_apr_may"}.issubset(df_merged.columns) else 0,
        (df_merged["number_booked_june"] > df_merged["maxbookees_june"]).sum()
        if {"number_booked_june","maxbookees_june"}.issubset(df_merged.columns) else 0
    ]
})


In [11]:
scaler = StandardScaler()
scaled_cols = ["maxbookees_apr_may", "number_booked_apr_may", "price_(inr)_apr_may"]

scaled_cols = [c for c in scaled_cols if c in df_merged.columns]
df_merged[scaled_cols] = scaler.fit_transform(df_merged[scaled_cols])


In [12]:
df_merged.to_csv(
    r"C:\Users\HP\Downloads\Fitness_Classes_Data\cleaned_preprocessed_dataset.csv",
    index=False
)

quality_report.to_csv(
    r"C:\Users\HP\Downloads\Fitness_Classes_Data\data_quality_report.csv",
    index=False
)


In [13]:
df_apr_may["bookingenddatetime_(month_/_day_/_year)"] = pd.to_datetime(
    df_apr_may["bookingenddatetime_(month_/_day_/_year)"],
    format="%m/%d/%Y",
    errors="coerce"
)

df_june["bookingenddatetime_(month_/_day_/_year)"] = pd.to_datetime(
    df_june["bookingenddatetime_(month_/_day_/_year)"],
    format="%m/%d/%Y",
    errors="coerce"
)


In [14]:
df_apr_may["bookingstarttime"] = pd.to_datetime(
    df_apr_may["bookingstarttime"],
    format="mixed",
    errors="coerce"
).dt.time

df_june["bookingstarttime"] = pd.to_datetime(
    df_june["bookingstarttime"],
    format="mixed",
    errors="coerce"
).dt.time


In [15]:
df_apr_may = df_apr_may.dropna(subset=["bookingenddatetime_(month_/_day_/_year)", "bookingstarttime"])
df_june = df_june.dropna(subset=["bookingenddatetime_(month_/_day_/_year)", "bookingstarttime"])


In [17]:
df_apr_may.dtypes
df_june.dtypes


activitysiteid                                     object
activitydescription                                object
bookingenddatetime_(month_/_day_/_year)    datetime64[ns]
bookingstarttime                                   object
maxbookees                                          int64
number_booked                                       int64
price_(inr)                                         int64
dtype: object

In [18]:
df_apr_may[[
    "bookingenddatetime_(month_/_day_/_year)",
    "bookingstarttime"
]].head()


,bookingenddatetime_(month_/_day_/_year),bookingstarttime


In [19]:
df_apr_may[
    df_apr_may["bookingenddatetime_(month_/_day_/_year)"].isna()
].head()


,activitysiteid,activitydescription,bookingenddatetime_(month_/_day_/_year),bookingstarttime,maxbookees,number_booked,price_(inr)


In [20]:
df_apr_may[["maxbookees", "number_booked", "price_(inr)"]].describe()


,maxbookees,number_booked,price_(inr)
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [21]:
df_merged.shape
df_merged.head()


,activitysiteid,activitydescription,bookingenddatetime_(month_/_day_/_year)_apr_may,bookingstarttime_apr_may,maxbookees_apr_may,number_booked_apr_may,price_(inr)_apr_may,bookingenddatetime_(month_/_day_/_year)_june,bookingstarttime_june,maxbookees_june,number_booked_june,price_(inr)_june
0,BRP,20:20:20 10-11 Am,2018-04-03,10:00:00,0.239673,1.528272,-1.731935,2018-06-05,10:00:00,35.0,25.0,499.0
1,BRP,20:20:20 10-11 Am,2018-04-03,10:00:00,0.239673,1.528272,-1.731935,2018-06-12,10:00:00,35.0,23.0,499.0
2,BRP,20:20:20 10-11 Am,2018-04-03,10:00:00,0.239673,1.528272,-1.731935,2018-06-19,10:00:00,35.0,21.0,499.0
3,BRP,20:20:20 10-11 Am,2018-04-03,10:00:00,0.239673,1.528272,-1.731935,2018-06-26,10:00:00,35.0,18.0,499.0
4,BRP,20:20:20 10-11 Am,2018-04-10,10:00:00,0.239673,0.891390,-1.731935,2018-06-05,10:00:00,35.0,25.0,499.0


In [22]:
quality_report


,check,value
0,total_records,9793
1,total_missing_values,750
2,duplicate_records,0
3,overbooked_apr_may,71
4,overbooked_june,32


In [23]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9793 entries, 0 to 9792
Data columns (total 12 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   activitysiteid                                   9793 non-null   object        
 1   activitydescription                              9793 non-null   object        
 2   bookingenddatetime_(month_/_day_/_year)_apr_may  9741 non-null   datetime64[ns]
 3   bookingstarttime_apr_may                         9741 non-null   object        
 4   maxbookees_apr_may                               9741 non-null   float64       
 5   number_booked_apr_may                            9741 non-null   float64       
 6   price_(inr)_apr_may                              9741 non-null   float64       
 7   bookingenddatetime_(month_/_day_/_year)_june     9695 non-null   datetime64[ns]
 8   bookingstarttime_june                 